# Testing C++ code in python
Jan 21, 2021 : Build 3d code and test

In [3]:
import numpy as np

## Multi-channel batch example

### Save image for C++ code

In [4]:
xsize,ysize,zsize=64,64,64
batch_size,num_channels=20,2

In [ ]:
### Create image data for 3d and store to numpy

# fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/3d_images_1000_64cube.npy'
# a1=np.load(fname)
# a1=np.expand_dims(a1,axis=1)
# np.save('data/3d_data/gen_images.npy',a1[:100])

In [ ]:
# # ### Save .npy file to csv for C++ code to handle
# fname='data/3d_data/gen_images.npy'
# a1=np.load(fname)
# print(a1.shape)

# a2=a1[0:batch_size*num_channels,0,:xsize,:ysize,:zsize].reshape(batch_size,num_channels,xsize,ysize,zsize)
# print(a2.shape)
# np.savetxt('data/3d_data/images1.csv',a2.flatten(),delimiter=',',newline=',')

# # ### Save second image
# a2=a1[batch_size*num_channels:2*batch_size*num_channels,0,:xsize,:ysize,:zsize].reshape(batch_size,num_channels,xsize,ysize,zsize)
# print(a2.shape)
# np.savetxt('data/3d_data/images2.csv',a2.flatten(),delimiter=',',newline=',')

### Read input

In [5]:
fname='data/3d_data/images1.csv'
x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize,zsize)
print(x.shape)
# x

(20, 2, 64, 64, 64)


### Compute batch spectrum

In [6]:
## numpy code: 3d 
def f_radial_profile(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    z, y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    center=[]
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0, (z.max()-z.min())/2.0]) # compute centers
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2+ + (z - center[2])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile

def f_compute_spectrum(arr):
#     GLOBAL_MEAN=1.0
#     arr=((arr - GLOBAL_MEAN)/GLOBAL_MEAN)
    
    y1=np.fft.fftn(arr)
    y1=np.fft.fftshift(y1)
    y2=abs(y1)**2
    z1=f_radial_profile(y2)
    return z1
   
def f_compute_batch_spectrum(arr):
    batch_pk=np.array([f_compute_spectrum(i) for i in arr])
    return batch_pk

### Code ###
def f_image_spectrum(x,num_channels):
    '''
    Data has to be in the form (batch,channel,x,y)
    '''
    mean=[[] for i in range(num_channels)]    
    var=[[] for i in range(num_channels)]    

    for i in range(num_channels):
        arr=x[:,i,:,:,:]
#         print(i,arr.shape)
        batch_pk=f_compute_batch_spectrum(arr)
#         print(batch_pk)
        mean[i]=np.mean(batch_pk,axis=0)
        var[i]=np.var(batch_pk,axis=0)
    mean=np.array(mean)
    var=np.array(var)
    return mean,var



In [7]:
mean,var=f_image_spectrum(x,2)
print(mean.shape,var.shape)

(2, 55) (2, 55)


### Read from c++ files and compare

In [8]:
### Read c++ files
cp_ip_file='data/3d_data/op_spec_mean.csv'
# x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
z_cpp1=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(num_channels,-1)

cp_ip_file='data/3d_data/op_spec_var.csv'
# x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
z_cpp2=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(num_channels,-1)
# print(z_cpp1,'\n',z_cpp2)

In [9]:
z_cpp1

array([[1.05077e+10, 5.66667e+08, 3.31514e+08, 2.12512e+08, 1.53772e+08,
        1.20276e+08, 9.54566e+07, 7.59563e+07, 6.27660e+07, 5.20677e+07,
        4.27720e+07, 3.60353e+07, 2.92948e+07, 2.44590e+07, 2.02665e+07,
        1.66532e+07, 1.37725e+07, 1.14375e+07, 9.43736e+06, 7.77411e+06,
        6.38597e+06, 5.27820e+06, 4.36862e+06, 3.63396e+06, 2.98751e+06,
        2.49793e+06, 2.08677e+06, 1.77039e+06, 1.50979e+06, 1.29422e+06,
        1.13169e+06, 9.99766e+05, 8.40493e+05, 6.84134e+05, 5.49504e+05,
        4.51334e+05, 3.70909e+05, 3.12067e+05, 2.61670e+05, 2.21553e+05,
        1.89385e+05, 1.67627e+05, 1.42777e+05, 1.24694e+05, 1.04931e+05,
        8.55181e+04, 7.02793e+04, 5.79066e+04, 5.04855e+04, 4.55947e+04,
        4.06418e+04, 3.64911e+04, 3.39010e+04, 3.11390e+04, 3.45070e+04],
       [8.03750e+09, 4.09302e+08, 2.20548e+08, 1.36791e+08, 9.68522e+07,
        7.75061e+07, 6.13644e+07, 4.92064e+07, 4.07101e+07, 3.38963e+07,
        2.81802e+07, 2.36207e+07, 1.97477e+07, 1.6

In [10]:
mean

array([[1.05076752e+10, 5.66667132e+08, 3.31513508e+08, 2.12511912e+08,
        1.53771872e+08, 1.20275673e+08, 9.54565849e+07, 7.59562614e+07,
        6.27660323e+07, 5.20677236e+07, 4.27720268e+07, 3.60352884e+07,
        2.92948419e+07, 2.44590127e+07, 2.02664642e+07, 1.66531784e+07,
        1.37725480e+07, 1.14374986e+07, 9.43735563e+06, 7.77410872e+06,
        6.38596938e+06, 5.27819937e+06, 4.36862430e+06, 3.63395705e+06,
        2.98750562e+06, 2.49792779e+06, 2.08677485e+06, 1.77038863e+06,
        1.50979205e+06, 1.29422034e+06, 1.13169382e+06, 9.99766190e+05,
        8.40493367e+05, 6.84134160e+05, 5.49503942e+05, 4.51334285e+05,
        3.70908566e+05, 3.12066713e+05, 2.61670465e+05, 2.21552931e+05,
        1.89384774e+05, 1.67626822e+05, 1.42776504e+05, 1.24693650e+05,
        1.04930732e+05, 8.55181103e+04, 7.02792723e+04, 5.79066102e+04,
        5.04854527e+04, 4.55946552e+04, 4.06418330e+04, 3.64910609e+04,
        3.39010196e+04, 3.11390260e+04, 3.45070232e+04],
       

In [11]:
### Check arrays
print(np.allclose(mean,z_cpp1[:,:mean.shape[1]],rtol=1e-4,atol=1e-8))
print(np.allclose(var,z_cpp2[:,:var.shape[1]],rtol=1e-4,atol=1e-8))


True
True


## Testing spectral loss

In [12]:
fname='data/3d_data/images1.csv'
x1=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize,zsize)
fname='data/3d_data/images2.csv'
x2=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize,zsize)


In [13]:
x1.shape,x2.shape,np.mean(x1),np.mean(x2)

((20, 2, 64, 64, 64),
 (20, 2, 64, 64, 64),
 0.990128432953664,
 0.9988072654821961)

In [16]:
mean1,var1=f_image_spectrum(x1,num_channels)
mean2,var2=f_image_spectrum(x2,num_channels)
print(mean1.shape,var1.shape)

(2, 55) (2, 55)


In [15]:
k_crop=int(xsize/2) ## =32
np.log(np.mean(np.square(mean1[:,:k_crop]-mean2[:,:k_crop]))),np.log(np.mean(np.square(var1[:,:k_crop]-var2[:,:k_crop])))

(37.57350187248992, 82.15832780120526)

### Conclusion:
Jan 21, 2021

Results matches c++ code output exactly, both spectra and losses \
One caveat: This code varies from the pytorch code in one way: the module f_radial_profile has a [1:-1] at the end there, but not here.

#### Update 
